# Matching Engine Skeleton

A basic matching engine to reconstruct fills from order data.

**Note**: This is a simplified version. Production implementation would need:
- Proper decimal handling
- All order types (triggers, TWAP, etc.)
- Self-trade prevention
- Reduce-only logic
- Position limits

In [ ]:
from decimal import Decimal
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from collections import defaultdict
from sortedcontainers import SortedDict
import json
from pathlib import Path

## Data Classes

In [ ]:
@dataclass
class Order:
    oid: int
    user: str
    asset: int
    side: str  # 'buy' or 'sell'
    price: Decimal
    size: Decimal
    remaining: Decimal
    tif: str  # 'Alo', 'Ioc', 'Gtc'
    reduce_only: bool
    timestamp: str

@dataclass
class Fill:
    timestamp: str
    asset: int
    price: Decimal
    size: Decimal
    buyer: str
    seller: str
    taker: str
    maker: str
    taker_oid: int
    maker_oid: int

## Order Book

In [ ]:
class OrderBook:
    def __init__(self, asset_id: int):
        self.asset_id = asset_id
        # Bids: highest price first (descending)
        self.bids: SortedDict[Decimal, List[Order]] = SortedDict(lambda x: -x)
        # Asks: lowest price first (ascending)
        self.asks: SortedDict[Decimal, List[Order]] = SortedDict()
        # Order lookup
        self.orders: Dict[int, Order] = {}
    
    def best_bid(self) -> Optional[Decimal]:
        return self.bids.peekitem(0)[0] if self.bids else None
    
    def best_ask(self) -> Optional[Decimal]:
        return self.asks.peekitem(0)[0] if self.asks else None
    
    def add_order(self, order: Order):
        book = self.bids if order.side == 'buy' else self.asks
        if order.price not in book:
            book[order.price] = []
        book[order.price].append(order)
        self.orders[order.oid] = order
    
    def remove_order(self, oid: int):
        if oid not in self.orders:
            return
        order = self.orders[oid]
        book = self.bids if order.side == 'buy' else self.asks
        if order.price in book:
            book[order.price] = [o for o in book[order.price] if o.oid != oid]
            if not book[order.price]:
                del book[order.price]
        del self.orders[oid]
    
    def would_cross(self, side: str, price: Decimal) -> bool:
        if side == 'buy' and self.asks:
            return price >= self.best_ask()
        if side == 'sell' and self.bids:
            return price <= self.best_bid()
        return False

## Matching Engine

In [ ]:
class MatchingEngine:
    def __init__(self):
        self.books: Dict[int, OrderBook] = defaultdict(lambda: OrderBook(0))
        self.fills: List[Fill] = []
        self.next_oid = 1
    
    def process_order(self, user: str, asset: int, side: str, price: str, 
                      size: str, tif: str, reduce_only: bool, timestamp: str) -> List[Fill]:
        """
        Process an incoming order.
        Returns list of fills generated.
        """
        price = Decimal(price)
        size = Decimal(size)
        book = self.books[asset]
        fills = []
        
        # Create order
        order = Order(
            oid=self.next_oid,
            user=user,
            asset=asset,
            side=side,
            price=price,
            size=size,
            remaining=size,
            tif=tif,
            reduce_only=reduce_only,
            timestamp=timestamp
        )
        self.next_oid += 1
        
        # ALO: reject if would cross
        if tif == 'Alo':
            if book.would_cross(side, price):
                return []  # Rejected
            book.add_order(order)
            return []
        
        # IOC or GTC: try to match
        fills = self._match(order, book, timestamp)
        
        # GTC: add remaining to book
        if tif == 'Gtc' and order.remaining > 0:
            book.add_order(order)
        
        # IOC: any remaining is cancelled
        
        return fills
    
    def _match(self, order: Order, book: OrderBook, timestamp: str) -> List[Fill]:
        """Match order against resting orders"""
        fills = []
        opposite = book.asks if order.side == 'buy' else book.bids
        
        while order.remaining > 0 and opposite:
            best_price, resting_orders = opposite.peekitem(0)
            
            # Check if price crosses
            if order.side == 'buy' and order.price < best_price:
                break
            if order.side == 'sell' and order.price > best_price:
                break
            
            # Match against resting orders at this price (FIFO)
            for resting in resting_orders[:]:
                fill_size = min(order.remaining, resting.remaining)
                
                fill = Fill(
                    timestamp=timestamp,
                    asset=order.asset,
                    price=best_price,  # Price improvement for taker
                    size=fill_size,
                    buyer=order.user if order.side == 'buy' else resting.user,
                    seller=resting.user if order.side == 'buy' else order.user,
                    taker=order.user,
                    maker=resting.user,
                    taker_oid=order.oid,
                    maker_oid=resting.oid
                )
                fills.append(fill)
                self.fills.append(fill)
                
                order.remaining -= fill_size
                resting.remaining -= fill_size
                
                # Remove filled resting order
                if resting.remaining == 0:
                    book.remove_order(resting.oid)
                
                if order.remaining == 0:
                    break
            
            # Remove empty price level
            if best_price in opposite and not opposite[best_price]:
                del opposite[best_price]
        
        return fills
    
    def process_cancel(self, asset: int, oid: int):
        """Cancel an order by ID"""
        book = self.books[asset]
        book.remove_order(oid)

## Test the Engine

In [ ]:
# Create engine
engine = MatchingEngine()

# Add some resting orders (ALO = maker only)
engine.process_order('maker1', 0, 'sell', '100.00', '1.0', 'Alo', False, '2023-01-01T00:00:00')
engine.process_order('maker2', 0, 'sell', '100.50', '2.0', 'Alo', False, '2023-01-01T00:00:01')
engine.process_order('maker3', 0, 'buy', '99.00', '1.5', 'Alo', False, '2023-01-01T00:00:02')

print(f"Best bid: {engine.books[0].best_bid()}")
print(f"Best ask: {engine.books[0].best_ask()}")

In [ ]:
# Send IOC buy order that crosses the spread
fills = engine.process_order('taker1', 0, 'buy', '100.50', '1.5', 'Ioc', False, '2023-01-01T00:00:03')

print(f"Generated {len(fills)} fills:")
for fill in fills:
    print(f"  {fill.buyer} bought {fill.size} from {fill.seller} @ {fill.price}")

In [ ]:
# Check book state after
print(f"Best bid: {engine.books[0].best_bid()}")
print(f"Best ask: {engine.books[0].best_ask()}")
print(f"Total fills: {len(engine.fills)}")

## Replay Blocks Through Engine

In [ ]:
def replay_blocks(blocks, engine=None):
    """
    Replay blocks through matching engine to reconstruct fills.
    """
    if engine is None:
        engine = MatchingEngine()
    
    fills_generated = []
    
    for block in blocks:
        timestamp = block['header']['block_time']
        
        for tx in block.get('txs', []):
            user = tx.get('user')
            if not user:
                continue
            
            for action in tx.get('actions', []):
                action_type = action.get('type')
                
                if action_type == 'order':
                    for order in action.get('orders', []):
                        fills = engine.process_order(
                            user=user,
                            asset=order.get('a'),
                            side='buy' if order.get('b') else 'sell',
                            price=order.get('p'),
                            size=order.get('s'),
                            tif=order.get('t', {}).get('limit', {}).get('tif', 'Gtc'),
                            reduce_only=order.get('r', False),
                            timestamp=timestamp
                        )
                        fills_generated.extend(fills)
                
                elif action_type == 'cancel':
                    for cancel in action.get('cancels', []):
                        engine.process_cancel(
                            asset=cancel.get('a'),
                            oid=cancel.get('o')
                        )
    
    return engine, fills_generated

In [ ]:
# Load sample blocks
with open('../hyperliquid_samples/explorer_blocks/100000100.json') as f:
    blocks = json.load(f)

print(f"Loaded {len(blocks)} blocks")
print(f"Time range: {blocks[0]['header']['block_time']} to {blocks[-1]['header']['block_time']}")

In [ ]:
# Replay through engine
engine, fills = replay_blocks(blocks)

print(f"Generated {len(fills)} fills")
print(f"Active order books: {len(engine.books)}")

In [ ]:
# Show sample fills
if fills:
    print("\nSample Fills:")
    for fill in fills[:10]:
        print(f"  Asset {fill.asset}: {fill.buyer[:8]}... bought {fill.size} @ {fill.price} from {fill.seller[:8]}...")

## Limitations

This basic engine doesn't handle:
- **Order IDs**: Real system tracks OIDs from the exchange
- **Modify orders**: Need to update existing orders
- **Trigger orders**: Stop loss, take profit
- **TWAP orders**: Time-weighted average price
- **Reduce-only**: Check position before allowing fill
- **Self-trade prevention**: Reject if maker == taker
- **Liquidations**: Forced closes
- **Funding**: Periodic payments between longs/shorts